In [103]:
import requests, re, typing
from bs4 import BeautifulSoup as soup
import contextlib
from selenium import webdriver
import pandas as pd
from urllib.request import urlopen
import csv
from datetime import datetime
import requests 
import mechanize
import http.cookiejar as cookielib
import html2text
from datetime import date
from bs4 import BeautifulSoup
import datetime

#  1. Scraping Historical ( Petrol  or Diesel ) price

## https://www.mypetrolprice.com/

In [173]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver

In [174]:
headers={'User-Agent': 'Chrome/127.0.0.1:9666'}

In [175]:
search_link = 'https://www.mypetrolprice.com/3/Diesel-price-in-Mumbai?FuelType=1&LocationId=3'

In [176]:
chromedriver = r'C:\Users\pprajapati3\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Python 3.8\chromedriver.exe'

In [177]:
driver = webdriver.Chrome(chromedriver)
driver.get(search_link)
action = ActionChains(driver)

In [178]:
end_d="31-Jan-2020"

In [179]:
diesel=[]
Date=[]
inf=10000
for i in range(inf):
    
    value1=driver.find_elements_by_class_name('GVPrice')
    value2=driver.find_elements_by_class_name('HDday')
    value3=driver.find_elements_by_class_name('HDmonth')
    value4=driver.find_elements_by_class_name('HDyear')
    
    w=str(value2[0].text)+"-"+str(value3[0].text)+"-"+str(value4[0].text)
    x=str(value2[1].text)+"-"+str(value3[1].text)+"-"+str(value4[1].text)
    y=str(value2[2].text)+"-"+str(value3[2].text)+"-"+str(value4[2].text)
    z=str(value2[3].text)+"-"+str(value3[3].text)+"-"+str(value4[3].text)
   # print(w)
   # print(x)
   # print(y)
   # print(z)
    if w==end_d:
        diesel.append((value1[0].text))
        Date.append(w)
        break
    elif x==end_d:
        diesel.append((value1[0].text))
        diesel.append((value1[1].text))
        Date.append(w)
        Date.append(x)
        break
    elif y==end_d:
        diesel.append((value1[0].text))
        diesel.append((value1[1].text))
        diesel.append((value1[2].text))
        Date.append(w)
        Date.append(x)
        Date.append(y)
        
        break
    elif z==end_d:
        diesel.append((value1[0].text))
        diesel.append((value1[1].text))
        diesel.append((value1[2].text))
        diesel.append((value1[3].text))
        Date.append(w)
        Date.append(x)
        Date.append(y)
        Date.append(z)
        break
    else:
        diesel.append((value1[0].text))
        diesel.append((value1[1].text))
        diesel.append((value1[2].text))
        diesel.append((value1[3].text))
        Date.append(w)
        Date.append(x)
        Date.append(y)
        Date.append(z)
    secondLevelMenu = driver.find_element_by_xpath("//*[@id='BC_GV_CustomGridPager_NextButton']")
    if i==0:
        action.move_to_element(secondLevelMenu).perform()
        
    try:
        secondLevelMenu.click()
        
    except:
        continue 

### Download CSV file

In [180]:
data_price = pd.DataFrame({'Date': Date,'Diesel_price':diesel})
data_price_final=data_price.drop_duplicates(subset='Date',keep="last").reset_index()
data_price_final.to_csv("diesel_price.csv")

In [183]:
data_price_final.head(10)

,index,Date,Diesel_price
0,4,24-Feb-2020,₹ 67.76
1,5,21-Feb-2020,₹ 67.71
2,6,20-Feb-2020,₹ 67.71
3,7,19-Feb-2020,₹ 67.71
4,8,18-Feb-2020,₹ 67.71
5,9,17-Feb-2020,₹ 67.76
6,10,15-Feb-2020,₹ 67.84
7,11,14-Feb-2020,₹ 67.84
8,12,13-Feb-2020,₹ 67.89
9,13,12-Feb-2020,₹ 67.94


#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#  2. Scraping all Holidays for any year ( State wise ) 

##  https://www.officeholidays.com/

In [138]:
Years=[2018,2019,2020]
Day=[]
Date=[]
Holiday_name=[]
Type=[]
Comments=[]
Year=[]

In [139]:
for i in Years:
    url='https://www.officeholidays.com/countries/india/maharashtra/'+str(i)
    headers={'User-Agent': 'Chrome/80.0.3987.106'}
    response=requests.get(url,headers)
    print(i)
    if response.status_code==200:
        print('Source Page Fetched')
    else:
        print('Error Cant Fecth Source Fetched')
    soup=BeautifulSoup(response.content,'html.parser')
    stat_table=soup.find_all('table',class_='country-table')
    pp=stat_table[0]
    u=0
    for row in pp.find_all('tr'):
        u=u+1
        y=[]
        for cell in row.find_all('td'):
           # print(cell.text)
            y.append(cell.text)
        try:
            
            Day.append(y[0])
            Date.append(y[1])
            Holiday_name.append(y[2])
            Type.append(y[3])
            Comments.append(y[4])
        except:
            continue
      
  #  print(u)
    year1=[i for j in range(u-1)]
    Year.extend(year1)             

2018
Source Page Fetched
2019
Source Page Fetched
2020
Source Page Fetched


## Cunstruct CSV file for Extracted data

In [140]:
Holiday_data=pd.DataFrame({"Day": Day,'Date': Date,'Year': Year,'Holiday_name': Holiday_name,'Type': Type,'Commnet': Comments})

In [141]:
Holiday_data.head(5)

,Day,Date,Year,Holiday_name,Type,Commnet
0,Friday,Jan 26,2018,Republic Day,Public Holiday,
1,Tuesday,Feb 13,2018,Mahashivratri,Regional Holiday,Most states
2,Monday,Feb 19,2018,Chahhtrapati Shivaji Maharaj Jayanti,Regional Holiday,Maharashtra only
3,Friday,Mar 02,2018,Holi (Second Day),Regional Holiday,
4,Sunday,Mar 25,2018,Ram Navami,Regional Holiday,


### Download into CSV form

In [142]:
Holiday_data.to_csv('Holidays.csv')

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 3. Scraping Historical Wearther Data
1. *District Wise.*
2. *Hourly Level.*

## https://www.worldweatheronline.com/

### You can Scrape data from (year 2009) till Today's date 

In [143]:
from wwo_hist import retrieve_hist_data
import os

#### Set path for downloaded files.

In [144]:
os.chdir(r"C:\Users\pprajapati3")

#### Set Hourly interval

In [145]:
# Six hours difference
frequency = 6 

#### List of Districts

In [149]:
location_list = [ 'Pune','Delhi'] 

In [150]:
start_date = '15-Feb-2020'
end_date = '19-Feb-2020'
api_key = 'b03be65990b147e294273846202001'

hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                            store_df = True)



Retrieving weather data for Pune


Currently retrieving data for Pune: from 2020-02-15 to 2020-02-19
Time elapsed (hh:mm:ss.ms) 0:00:00.843205


export Pune completed!




Retrieving weather data for Delhi


Currently retrieving data for Delhi: from 2020-02-15 to 2020-02-19
Time elapsed (hh:mm:ss.ms) 0:00:00.844449


export Delhi completed!




#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 4. Scraping Future Weather data ( From today )
## https://www.accuweather.com/

### Count number of days from today_date to End_date

In [151]:
today_date=date(2020,2,21)
end_date=date(2020,3,31)
no_days=end_date-today_date
no_days=int(no_days.days+1)

In [152]:
Min_temp=[]
Max_temp=[]
Precipitation=[]
Sun_hours=[]

### Set Chrome Header for VPN Location  ( IP Address ) from (dovelopers tool of chrome ) :

In [153]:
headers={'User-Agent': 'Chrome/127.0.0.1:9666'}

### Scrape Data for all dates

In [154]:
for i in range(1,(no_days+1)):
    
    print("day : ",i)
    url='https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day='+str(i)
    print(url)
    
    
    headers={'User-Agent': 'Chrome/127.0.0.1:9666'}
    webpage = requests.get(url, headers=headers,timeout=100)
    print(webpage)
    soup=BeautifulSoup(webpage.content,'html.parser')
    
    stat_table=soup.find_all('p',class_='value')
    y=str(stat_table[0])
    y=y.replace('<p class="value">\n\t\t\t\t','')
    y=y.replace('°<span class="hi-lo-label">Hi</span>\n</p>','')
    # print(int(y))
    temp1= int(y)
    #print(temp1)
    
    y=str(stat_table[1])
    y=y.replace('<p class="value">\n\t\t\t\t','')
    
    y=y.replace('°<span class="hi-lo-label">Lo</span>\n</p>','')
    
    temp2= int(y)  
    #print(temp2)
    stat_table=soup.find_all('span',class_='section-content')
    pp=stat_table[0]
    
    
    stat_table=soup.find_all('div',class_='temperatures')
    stat_table=soup.find_all('span',class_='section-content last')
    y=str(stat_table[0])
    y=y.replace('<span class="section-content last">\n\t\t\t\t\t','')
    y=y.replace(' hr\n\t\t\t\t</span>','')
    y=y.replace(':','.')
    sun_hour=float(y)
    
    
    stat_table=soup.find_all('div',class_='list')
    pp=stat_table[0]
    op=0
    for row in pp.find_all('p'):
        p=row.text
        #print(p[4])
        if p[4]=='R':
            op=float(p[9:13])
    op1=0
    try:
    
        pp=stat_table[1]
        
        for row in pp.find_all('p'):
            p=row.text
        
            if p[4]=='R':
                op1=float(p[9:13])
                
    except:
        Max_temp.append(temp1)
        Min_temp.append(temp2)
        Precipitation.append(op+op1)
        Sun_hours.append(sun_hour)
        continue
        
    #print(op)
    #print(op1)
    Max_temp.append(temp1)
    Min_temp.append(temp2)
    Precipitation.append(op+op1)
    Sun_hours.append(sun_hour)

day :  1
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=1
<Response [200]>
day :  2
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=2
<Response [200]>
day :  3
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=3
<Response [200]>
day :  4
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=4
<Response [200]>
day :  5
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=5
<Response [200]>
day :  6
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=6
<Response [200]>
day :  7
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=7
<Response [200]>
day :  8
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=8
<Response [200]>
day :  9
https://www.accuweather.com/en/in/delhi/202396/daily-weather-forecast/202396?day=9
<Response [200]>
day :  10
https://w

In [155]:
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(no_days)]

In [156]:
data_file=pd.DataFrame({'Date':date_list,'Max_temp': Max_temp ,'Min_temp': Min_temp ,'Precipitation': Precipitation,'Sun_hours': Sun_hours })

In [157]:
data_file.head(10)

,Date,Max_temp,Min_temp,Precipitation,Sun_hours
0,2020-02-24 11:39:32.828403,26,13,0.0,11.26
1,2020-02-23 11:39:32.828403,26,14,0.0,11.28
2,2020-02-22 11:39:32.828403,27,14,0.0,11.29
3,2020-02-21 11:39:32.828403,28,15,0.0,11.31
4,2020-02-20 11:39:32.828403,28,17,0.0,11.33
5,2020-02-19 11:39:32.828403,28,16,2.0,11.34
6,2020-02-18 11:39:32.828403,28,16,0.0,11.36
7,2020-02-17 11:39:32.828403,28,16,0.0,11.37
8,2020-02-16 11:39:32.828403,28,16,1.0,11.39
9,2020-02-15 11:39:32.828403,27,16,1.0,11.41


#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------